# Classification of Matroidal Subdivisions of (3,8)-hypersimplex

In the following computations we classify matroidal subdivisions of $\Delta(3,8)$ using properties of their dual graphs. We study these subdivisions up to symmetry. This symmetry is induced by the action of the symmetric group $\mathfrak{S}_8$ on height functions $w\in\text{TGr}_{0}(3,8)\subset \mathbb{R}^{{[8]\choose 3}}$, where we think of $\mathfrak{S}_8$ as a subgroup of $\mathfrak{S}_{56}$.

That is, for $\sigma\in\mathfrak{S}_{8}$ and $\{i,j,k\}\in{[8]\choose 3}$, we take $\sigma\circ\{i,j,k\}=\{\sigma(i),\sigma(j),\sigma(k)\}$. In this way, we induce a permutation on the entries of $w$.

For each cone of $\text{TGr}_{0}(3,8)$, we have a representative from the interior. In other words, we have a representative for each symmetry class of matroidal subdivision.

For a complete explanation, see <a href="https://alco.centre-mersenne.org/articles/10.5802/alco.302/">Section 6</a>.

In [ ]:
using Oscar
using Combinatorics
pm = Polymake

In [ ]:
cd("..")

In [ ]:
include("src/inputData38.jl");
include("src/fileHandling.jl")
include("src/tscCoordRing.jl");
include("src/matroidalSubd.jl");
include("src/Bmaximal.jl");
include("src/simplifyIdeal.jl");

Let $w\in\text{TGr}_{0}(3,8)$. Then $w$ induces a matroidal subdivision of $\Delta(3,8)$, denoted $\mathcal{Q}(w)$. The tightspan, denoted $\text{TS}(w)$ is a polytopal complex whose faces are dual to $\mathcal{Q}(w)$. Each face of codimension $k$ of $TS(w)$ corresponds to a $k$ dimensonal face of $\mathcal{Q}(w)$.

Let $\Gamma(w)$ be the graph dual to the subdivision induced by $w$. That is, $\Gamma(w)$ has a vertex $v$ for each maximal cell $C_v$ of $\mathcal{Q}(w)$, and an edge between $v$ and $v^{\prime}$ if $C_v$ and $C_{^{\prime}}$ intersect in codimension one. Observe that $\Gamma(w)$ is the 1-skeleton of $\text{TS}(w)$.


**Variable notation**

```cone``` = cone of $\text{TGr}_0(3,8)$ corresponding to representative $w$ from interior.

```subd``` = $\mathcal{Q}(w)$.

```G``` = $\Gamma(w)$.

```Tmc``` = $\text{TS}(w)$.

```Mp``` = maximal polytopes of $\text{TS}(w)$ as sets of vertices of $\text{TS}(w)$. Note that these are saved as a matrix such that the $ith$ row corresponds to $ith$ polytope. 

```Mc``` = maximal cells of $\mathcal{Q}(w)$ as sets of vertices of $\Delta(3,8)$, saved as array of vectors of indices of vertices.

# Functions

**Function** ```use_edges```

*input* ```G```

*output* ```Set```

*Description* returns edges of ```G``` such that they are usable in subsequent functions.

In [ ]:
function use_edges(G)
    E = collect(edges(G))
    return EE = Vector([sort([dst(e),src(e)]) for e in E])
end

**Function**:```common_vertex``` 

*Input*: ```Mc```

*Output*: ```Set``` of indices of vertices of $\Delta(3,8)$.

*Description*: intersects vertex sets of maximal cells of $\mathcal{Q}(w)$, with the objective of finding a common basis of the matroids corresponding to these cells.

In [ ]:
function common_vertex(Mc)
    return reduce(intersect,[c for c in Mc])
end

**Function**: ```is_tree```

*Input*: ```Gra```, ```Tmc```

*Output*: ```Integer```$F=E-V+1$

*Description*: Checks if $\Gamma(w)$ is a tree

In [ ]:
function is_tree(G)
    return 0 == ne(G)-nv(G)+1 
end

**Function** ```f```

*input*: edge ``e``,vertices of supergraph ``A``

*output*: elements of ``E`` relabeled using indices starting at 1

*Description* relabels vertices in subset of edges such that we can compute induced subgraph without extra vertices

In [ ]:
function f(e, A)
    return [[i,j] for i in 1:length(A),j in 1:length(A) if A[i] == e[1] && A[j] == e[2]][1]
end

**Function**: ```leaf_cvp```

*Input*: ```Mc```, ```G```

*Output*: ```Set``` 

*Description*: Computes indices of common vertices maximal polytopes of $\mathcal{Q}(w)$ corresponding to faces of the subcomplex $\Sigma_{L}\subset \Sigma$ obtained by removing all leaves.

In [ ]:
function leaf_cvp(Mc,G)
    c = [Mc[i] for i in 1:nv(G) if length(neighbors(G,i))>1]
    return reduce(intersect, c)
end

**Function** ``find_leaves_edges``

*input*: ``E``, edges of graph

*output*: ``Set``

*Description* Identifies leaf edges on graph ``G``. To be applied itertively

In [ ]:
function find_leaves_edges(E)# E = edge set of graph
    vs = reduce(union,E)
    ls = [i for i in vs if length([e for e in E if i in e]) == 1]
    les = [e for e in E if e[1] in ls||e[2] in ls]
    return les
end

**Function** ``find_branches``

*input*: ``E``, edges of graph

*output*: ``Set``

*Description* iteratively identifies leaf edges on graph ``G``. 

In [ ]:
function find_branches(E)
    L = find_leaves_edges(E)
    B = vcat([],L)
    while !isempty(L)
        E = setdiff(E,L)
        L = find_leaves_edges(E)
        B = vcat(B,L)
    end
    return B
end

**Function**: ```branch_cvp```

*Input*: ```Mc```, ```G```

*Output*: ```Set``` 

*Description*: Computes indices of common vertices of maximal polytopes $\mathcal{Q}(w)$ corresponding to faces of the subcomplex $\Sigma_{B}\subset \Sigma$ obtained by removing all branches.

In [ ]:
function branch_cvp(Mc,G)
    E = use_edges(G)
    B = find_branches(E)
    vs = reduce(union, setdiff(E,B))
    return reduce(intersect,[Mc[i] for i in vs])
end

**Function** ``fin_expoed``

*Input*: ``G``,``Fins=`` fins on $\Gamma(w)$

*Output*: ``Set``

*Description*: Indentifies exposed vertices of fins

In [ ]:
function fin_exposed(G,Fins)
    E = use_edges(G)    
    Branches = find_branches(E)
    Not_Branches = setdiff(Mp,Branches)
    exposed = []
    for P in Fins
        PB = setdiff(Not_Branches,[P])
        int = intersect(reduce(union,PB),P)
        ex = [i for i in P if !(i in int)]
        exposed = vcat(exposed,ex)
    end
    return exposed    
end

**Function** ```fins_1```

*input*: ```G```,```Ts```,```Mp```

*Output*: ```Set```

*Description* Identifies fins of contact edgelength $1$ on ```G```.

In [ ]:
function fins_1(G,Ts,Mp)
    P2 = polyhedra_of_dim(Ts,2)
    Pinds = [[i for i in 1:length(vertices(Ts)) if vertices(Ts)[i] in vertices(P)] for P in P2]
    E = use_edges(G)
    NoBranch = setdiff(Mp,find_branches(E))
 
    finz = []
    for P in Pinds
        BodyP = setdiff(NoBranch,[P])
        Bint = intersect(reduce(union,BodyP),P)
        Es = [e for e in E if issubset(e,Bint)]
        if length(Es) == 1
            push!(finz,P)
        end  
    end  
    return finz
end

**Function** ```fin_1_cvp```

*input* : ```Mp```, ```Mc```, ```G```

*output*: ```Set```

*Description*: Computes indices of common vertices of cells of $\mathcal{Q}(w)$ corresponding to faces of the subcomplex $\Sigma_{F(1)}\subset\Sigma_{L}\subset\Sigma$ obtained by removing all leaves and fins with contact length $1$. 

In [ ]:
function fin_1_cvp(Mc,G,Ts,Mp)
    Fins = fins_1(G,Ts,Mp)
    vs = [i for i in 1:nv(G) if length(neighbors(G,i)) != 1] #restrict to non leaf vertices
    exposed = fin_exposed(G,Fins)
    return reduce(intersect,[Mc[i] for i in setdiff(vs,exposed)])
end

**Function** ```is_path```

*input*: ```Ograph```

*output*: ```bool```

*Description*: Checks if graph is isomorphic to a path. To be used to identify general fins

In [ ]:
function is_path(Ograph)
    E = use_edges(Ograph)
    gpath = graph_from_edges(Vector([[i,i+1] for i in 1:length(E)]))
    return is_isomorphic(gpath,Ograph)
end

**Function** ``fins``

*input*: ``G``,``Ts``,``Mp``

*output*: ``Set``, collection of fins (of any contanct length) of $\Gamma(w)$

*Description*: Find fins on dual graph of tight span

In [ ]:
function fins(G, Ts, Mp)
    P2 = polyhedra_of_dim(Ts, 2) 
    Pinds = [[i for i in 1:length(vertices(Ts)) if vertices(Ts)[i] in vertices(P)] for P in P2]
    E = use_edges(G)
    NoBranch = setdiff(Mp,find_branches(E))
    finz = []
    for P in Pinds
        BodyP = setdiff(NoBranch, [P])
        Bint = intersect(reduce(union, BodyP),P)
        Es = [e for e in E if issubset(e,Bint)]
        E2 = [f(e,Bint) for e in Es]  
        if 1 <= length(Es) <= length(P)-2 && is_path(graph_from_edges(E2, length(Bint)))
            push!(finz,P)
        end
    end
    return finz
end

**Function** ``fins_tree``

``input``: ``G``,``Mp``,``Ts``

``output``: ``(bool,Graph)``

``Description``: Checks if $\Sigma_{F}\subset\Sigma_{L}\subset\Sigma$ obtained by removing all leaves and fins is a tree. Returns induced subgraph.

In [ ]:
function fins_tree(G,Ts,Mp)
    E = use_edges(G)
    Fins = fins(G,Ts,Mp) #compute fins
    exposed = fin_exposed(G,Fins) #compute exposed vertices of fins
    vs = [i for i in 1:nv(G) if length(neighbors(G,i))>1] #restrict to non leaf vertices
    internal = setdiff(vs,exposed) #compute internal vertices
    bes = [e for e in E if issubset(e,setdiff(vs,exposed))] #comute internal edges
    Es2 = [f(e,setdiff(vs,exposed)) for e in bes] #relabel vertices for induced subgraph
    GG = graph_from_edges(Es2) #compute induced subgraph
    return is_tree(GG),GG
end  

 # Classes of subdivision
 
 We will use the functions defined above to sort matroidal subdivsions of $\Delta(3,8)$ into 6 combinatorial types.
 
 **G1**: Maximal cells of $\mathcal{Q}(w)$ share a common vertex.
 
 **G2**: $\Gamma(w)$ is a tree.
 
 **G3**: The maximal cells of $\mathcal{Q}(w)$ corresponding to the subcomplex of $\text{TS}(w)$ obtained by removing all leaves share a common vertex.
 
 **G4**: The maximal cells of $\mathcal{Q}(w)$ corresponding to the subcomplex of $\text{TS}(w)$ obtained by removing all branches share a common vertex.
 
 **G5**: The maximal cells of $\mathcal{Q}(w)$ corresponding to the subcomplex of the $TS(w)$ obtained by first removing all leaves, and then removing all fins that intersect the body in a single edge, share a common vertex.
 
 **G6**: The maximal cells of $\mathcal{Q}(w)$ corresponding to subcomplex of $TS(w)$ obtained by removing all leaves and fins is a tree.
 
 Note that these categories are not mutually exclusive. We require that if a subdivision corresponding to $w$ is in $Gj$, then it is not in $Gi$ for $i<j$.
 
 Sorting matroid subdivisions up to symmetry in this way gives us a complete classification of all combinatorial types, for which we develop theory to prove properites about inverse limits.

# Examples

We compute some applications of the above functions.

In [ ]:
d38 = hypersimplex(3,8)

***$G_1$ Common vertex property***

In [ ]:
w = [-7, -33, 6, 6, 26, 25, -11, -2, -2, 18, 17, 2, 2, 22, 21, -42, -4, -5, -4, -5, -30, -32, 7, 7, -33, -34, -34, -34, 1, 0, 2, 40, 39, 40, 39, -31, 3, 3, -7, -8, -6, 32, 31, 32, 31, -39, -2, 36, 35, 36, 35, -35, -68, -54, -16, -16]
S = subdivision_of_points(d38,-w)#compute subdivision

Tmc = S.pm_subdivision.TIGHT_SPAN
Mp = Tmc.MAXIMAL_POLYTOPES#obtain maximal polytopes
Mp = [Vector(pm.row(Mp,i)) for i in 1:nrows(Mp)]#convert maximal polytopes to usable type
Mc = maximal_cells(S)#maximal cells of S
G = S.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
G = Graph{Undirected}(G.ADJACENCY)
visualize(G)

In [ ]:
common_vertex(Mc)

**$G_2$ Tree**

In [ ]:
w = [12, 5, 5, -86, 5, 19, 5, 5, 19, 5, -86, -2, 12, -2, 12, 12, -2, 12, 12, 26, 12, 12, 12, 26, -93, 26, 5, 19, 5, -86, 19, 5, 19, 19, 33, 19, 5, -86, 5, 19, 19, 5, 19, 19, 33, 19, 12, -2, 12, 12, 26, 12, -23, -114, -23, -9]
S = subdivision_of_points(d38,-w)#compute subdivision

Tmc = S.pm_subdivision.TIGHT_SPAN
Mp = Tmc.MAXIMAL_POLYTOPES#obtain maximal polytopes
Mp = [Vector(pm.row(Mp,i)) for i in 1:nrows(Mp)]#convert maximal polytopes to usable type
Mc = maximal_cells(S)#maximal cells of S
G = S.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
G = Graph{Undirected}(G.ADJACENCY)
visualize(G)

In [ ]:
common_vertex(Mc)

In [ ]:
is_tree(G)

**$G_3$ Leaf common vertex propety**

In [ ]:
w = [17, 21, -4, -11, 17, 20, 34, 9, 2, -30, 33, -62, -9, 34, -23, -34, 9, -48, 2, -10, 33, -46, 24, 17, -35, -32, 28, 21, -31, -28, -19, 24, 27, 17, 20, -47, 41, 34, -18, -15, -36, -23, 40, -30, 33, -19, -2, 41, -16, 34, -23, -15, -36, -3, 40, 33]
S = subdivision_of_points(d38,-w)#compute subdivision

Tmc = S.pm_subdivision.TIGHT_SPAN
Mp = Tmc.MAXIMAL_POLYTOPES#obtain maximal polytopes
Mp = [Vector(pm.row(Mp,i)) for i in 1:nrows(Mp)]#convert maximal polytopes to usable type
Mc = maximal_cells(S)#maximal cells of S
G = S.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
G = Graph{Undirected}(G.ADJACENCY)
visualize(G)

In [ ]:
common_vertex(Mc)
println("common vertex: ", common_vertex(Mc))

In [ ]:
is_tree(G)

In [ ]:
leaf_cvp(Mc,G)

**$G_4$ Branch common vertex property**

In [ ]:
w = [1, 8, 1, -20, 8, 8, 8, -20, -20, 8, 8, 8, -13, 15, 15, -20, 8, 8, -13, -13, 15, 8, -20, 1, 8, 8, 8, 8, -20, -20, 1, 8, 8, 8, 8, -20, -13, 8, 15, 15, -20, -13, -13, 8, 8, 15, 8, 15, 15, 15, 15, -118, 8, 8, 15, 15]
S = subdivision_of_points(d38,-w)#compute subdivision

Tmc = S.pm_subdivision.TIGHT_SPAN
Mp = Tmc.MAXIMAL_POLYTOPES#obtain maximal polytopes
Mp = [Vector(pm.row(Mp,i)) for i in 1:nrows(Mp)]#convert maximal polytopes to usable type
Mc = maximal_cells(S)#maximal cells of S
G = S.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
G = Graph{Undirected}(G.ADJACENCY)
visualize(G)

In [ ]:
common_vertex(Mc)

In [ ]:
is_tree(G)

In [ ]:
leaf_cvp(Mc,G)

In [ ]:
branch_cvp(Mc,G)

**$G_5$ Fin common vertex property**

In [ ]:
w = [-4, -22, 0, -36, 15, 15, -9, -32, 7, 28, 28, -5, -11, -35, -35, 11, 32, 32, 26, 26, -31, -4, -27, -3, 18, 18, 0, -21, 15, 15, 1, 22, 22, 16, 16, -56, -47, 7, 28, 28, -16, 5, 5, -1, -1, -28, 11, 32, 32, 26, 26, -31, -27, -27, -24, -30]
S = subdivision_of_points(d38,-w)#compute subdivision

Tmc = S.pm_subdivision.TIGHT_SPAN
Ts = polyhedral_complex(Tmc)
Mp = Tmc.MAXIMAL_POLYTOPES#obtain maximal polytopes
Mp = [Vector{Int64}(pm.row(Mp,i)) for i in 1:nrows(Mp)]#convert maximal polytopes to usable type
Mc = maximal_cells(S)#maximal cells of S
G = S.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
G = Graph{Undirected}(G.ADJACENCY)
visualize(G)

In [ ]:
common_vertex(Mc)

In [ ]:
is_tree(G)

In [ ]:
leaf_cvp(Mc,G)

In [ ]:
branch_cvp(Mc,G)

In [ ]:
fin_1_cvp(Mc,G,Ts,Mp)

***$G_6$ Fin tree complex***

In [ ]:
w = [-18, -46, 17, 17, 59, 52, -18, 24, 24, -284, -186, 17, 17, 59, 52, -67, 101, 94, 101, 94, -109, -46, 17, 17, 59, 52, -137, -137, 10, 3, -95, 73, 66, 73, 66, -102, 17, 17, 59, 52, -67, 101, 94, 101, 94, -109, -95, 73, 66, 73, 66, -102, -116, -123, -60, -60]
S = subdivision_of_points(d38,-w)#compute subdivision

Tmc = S.pm_subdivision.TIGHT_SPAN
Ts = polyhedral_complex(Tmc)
Mp = Tmc.MAXIMAL_POLYTOPES#obtain maximal polytopes
Mp = [Vector(pm.row(Mp,i)) for i in 1:nrows(Mp)]#convert maximal polytopes to usable type
Mc = maximal_cells(S)#maximal cells of S
G = S.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
G = Graph{Undirected}(G.ADJACENCY)
visualize(G)

In [ ]:
common_vertex(Mc)

In [ ]:
is_tree(G)

In [ ]:
leaf_cvp(Mc,G)

In [ ]:
branch_cvp(Mc,G)

In [ ]:
fin_1_cvp(Mc,G,Ts,Mp)

In [ ]:
fins_tree(G,Ts,Mp)

In [ ]:
visualize(fins_tree(G,Ts,Mp)[2])

# Complete classification of subdivisions


The code below loops through all representative height functions for cones, checking for the correct property, and adding it to the appropriate set $G_{1},G_{2},G_{3},G_{4},G_{5},G_{6}$.

**warning :** Fully implementing this classification is time consuming.

In [ ]:
codim0 = file2SetVectors("allRepsByCodim/codim_0.dat")
codim1 = file2SetVectors("allRepsByCodim/codim_1.dat")
codim2 = file2SetVectors("allRepsByCodim/codim_2.dat")
codim3 = file2SetVectors("allRepsByCodim/codim_3.dat")
codim4 = file2SetVectors("allRepsByCodim/codim_4.dat")
codim5 = file2SetVectors("allRepsByCodim/codim_5.dat")
codim6 = file2SetVectors("allRepsByCodim/codim_6.dat")
codim7 = file2SetVectors("allRepsByCodim/codim_7.dat")
all_cones = union(codim0,codim1,codim2,codim3,codim4,codim5,codim6,codim7)

In [ ]:
for i in 1:length(all_cones)
 
    w = all_cones[i]
 
    S = subdivision_of_points(d38,-w)#compute subdivision
    
    Tmc = S.pm_subdivision.TIGHT_SPAN
    Ts = polyhedral_complex(Tmc)
    
    Mp = Tmc.MAXIMAL_POLYTOPES#obtain maximal polytopes
    Mp = [Vector{Int64}(pm.row(Mp,i)) for i in 1:nrows(Mp)]#convert maximal polytopes to usable type
    
    Mc = maximal_cells(S)#maximal cells of S
    
    G = S.pm_subdivision.POLYHEDRAL_COMPLEX.DUAL_GRAPH
    G = Graph{Undirected}(G.ADJACENCY)
    
    if length(common_vertex(Mc))>0
       # println(i," cvp!")
        
        txtcvp = vec2String(w)
        open("notebooks/new_save_grand_scheme/cvp2.dat", "a")do file
        write(file, txtcvp,"\n")
        end
        continue
        
    elseif is_tree(G) #F = E -V + 1
        # println(i," tree!")
        
        txttree = vec2String(w)
        open("notebooks/new_save_grand_scheme/tree2.dat", "a")do file
        write(file, txttree,"\n")
        end
        continue

    elseif length(leaf_cvp(Mc,G))>0
        #println(i," leaf!")
        
        txtleaf = vec2String(w)
        open("notebooks/new_save_grand_scheme/leaf_cvp2.dat", "a")do file
        write(file, txtleaf,"\n")
        end
        continue
    
    elseif length(branch_cvp(Mc,G))>0
        #println(i," branch!")
     
        txtbranch = vec2String(w)
        open("notebooks/new_save_grand_scheme/branch_cvp2.dat", "a")do file
        write(file, txtbranch,"\n")
        end
        continue
    
    elseif length(fin_1_cvp(Mc,G,Ts,Mp))>0
        #println(i," fin1!")
     
        txtfin = vec2String(w)
        open("notebooks/new_save_grand_scheme/fin_cvp2.dat", "a")do file
        write(file, txtfin,"\n")
            end
        continue
    elseif fins_tree(G,Ts,Mp)[1]
        #println(i," fin2!")
      
        txtfintree = vec2String(w)
        open("notebooks/new_save_grand_scheme/fin_tree2.dat", "a")do file
        write(file, txtfintree,"\n")
        end
        continue
        
    else
        #println(i," uh oh")
        
        txtcatch = vec2String(w)
        open("notebooks/new_save_grand_scheme/catch2.dat", "a")do file #this should be empty
        write(file, txtcatch,"\n")
            end
        continue
        
        
    end

end

print("DONE :)")

In [ ]:
a1 = vec(readlines("notebooks/new_save_grand_scheme/cvp2.dat"))
a2 = vec(readlines("notebooks/new_save_grand_scheme/tree2.dat"))
a3 = vec(readlines("notebooks/new_save_grand_scheme/leaf_cvp2.dat"))
a4 = vec(readlines("notebooks/new_save_grand_scheme/branch_cvp2.dat"))
a5 = vec(readlines("notebooks/new_save_grand_scheme/fin_cvp2.dat"))
a6 = vec(readlines("notebooks/new_save_grand_scheme/fin_tree2.dat"))

In [ ]:
length(a1)#13641 in paper

In [ ]:
length(a2)#on point 

In [ ]:
length(a3)#on point 

In [ ]:
length(a4)#on point 

In [ ]:
length(a5)#on point

In [ ]:
length(a6)#on point

In [ ]:
length(a1) + length(a2) + length(a3) + length(a4) + length(a5) + length(a6)  == length(all_cones) #sorting exhaustive

In [ ]:
A = subsets([a1,a2,a3,a4,a5,a6],2)#check for disjoint

In [ ]:
[intersect(b[1],b[2]) for b in A]